In [1]:
%%capture
!pip install dash
!pip install LibRecommender

In [2]:
import re
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
from itertools import product
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import display, clear_output
from libreco.data import split_by_ratio_chrono, DatasetPure, DatasetFeat
from libreco.data import DataInfo, DatasetFeat, split_by_ratio_chrono
from libreco.algorithms import NCF, DeepFM
from libreco.evaluation import evaluate
import tensorflow as tf
import dash
from dash import dcc, html, callback, Input, Output, State, Dash
tf.compat.v1.reset_default_graph()
tf.get_logger().setLevel('ERROR')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

Instructions for updating:
non-resource variables are not supported in the long term


In [3]:
!unzip '/content/deepfm_2.zip'

Archive:  /content/deepfm_2.zip
replace deepfm_data_info.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [4]:
data_info = DataInfo.load("/content/", model_name="deepfm")

In [5]:
model = DeepFM.load(path="/content/", model_name="deepfm", data_info=data_info, manual=True)

total params: 177,405 | embedding params: 163,018 | network params: 14,387


In [6]:
api_key = 'ccce010'

In [7]:
links = pd.read_csv("/content/Links_processed.csv")
movie_ratings = pd.read_csv("/content/movies_rating_df.csv")
movies_genres_df_new = pd.read_csv("/content/movies_genres_df.csv")
movie_ratings_df = pd.read_csv("/content/movies_rating_df.csv")
movies_genres_df_new['genres'] = [re.findall(r'\b[a-zA-Z0-9\']+\b', x) for x in movies_genres_df_new['genres']]

In [8]:
def top_similar_movies_based_on_rating(movie_ratings_df, title, top_n=50):
    movie_id = movie_ratings_df.loc[movie_ratings_df['title'] == title, 'movieId'].values[0]
    user_item_matrix = movie_ratings_df.pivot_table(index='userId', columns='movieId', values='rating').fillna(0)
    item_similarity = cosine_similarity(user_item_matrix.T)
    item_similarity_df = pd.DataFrame(item_similarity, index=user_item_matrix.columns, columns=user_item_matrix.columns)
    similar_movies = item_similarity_df.loc[movie_id].sort_values(ascending=False)
    top_movies_ids = similar_movies.index[1:top_n+1]
    top_movies_titles = [movie_ratings_df.loc[movie_ratings_df['movieId'] == mid, 'title'].values[0] for mid in top_movies_ids]
    return top_movies_titles

In [9]:
def top_similar_movies_based_on_genres(movies_df, movie_title, top_n=10):
    mlb = MultiLabelBinarizer()
    genres_encoded = mlb.fit_transform(movies_df['genres'])
    genres_df = pd.DataFrame(genres_encoded, columns=mlb.classes_)
    genre_similarity = cosine_similarity(genres_df)
    genre_similarity_df = pd.DataFrame(genre_similarity, index=movies_df['movieId'], columns=movies_df['movieId'])
    movie_id = movies_df.loc[movies_df['title'] == movie_title, 'movieId'].values[0]
    similar_movies_genre = genre_similarity_df.loc[movie_id].sort_values(ascending=False)
    top_movies_ids = similar_movies_genre.index[1:top_n+1]
    top_movies_titles = [movies_df.loc[movies_df['movieId'] == mid, 'title'].values[0] for mid in top_movies_ids]
    return top_movies_titles

In [10]:
genre_recommendations = []
rating_recommendations = []

def initialize_recommendations(movie_title):
    genre_recommendations = top_similar_movies_based_on_genres(movies_genres_df_new, movie_title, top_n=20)
    rating_recommendations = top_similar_movies_based_on_rating(movie_ratings_df, movie_title, top_n=20)
    return genre_recommendations, rating_recommendations

In [11]:
def fetch_movie_poster(imdb_id):
    api_key = 'ccce010'
    url = f'http://www.omdbapi.com/?i={imdb_id}&apikey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        movie_data = response.json()
        return movie_data.get('Poster')
    else:
        return None

In [12]:
def fetch_movie_title(imdb_id):
    api_key = 'ccce010'
    url = f'http://www.omdbapi.com/?i={imdb_id}&apikey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        movie_data = response.json()
        return movie_data.get('Title')
    else:
        return None

In [13]:
# Importing external stylesheets with Font Awesome
external_stylesheets = ['https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css',
                        'https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.4/css/all.min.css']

# Creating the Dash app
app = Dash(__name__, external_stylesheets=external_stylesheets)

# Define the layout for the Main page
main_page = html.Div([
    html.Div([
        html.Img(src="/assets/output-onlinepngtools.png", style={'marginTop': '100px', 'width': '200px', 'display': 'block', 'margin-left': 'auto', 'margin-right': 'auto'}),
        html.H1('MovieLens Recommendations', className='text-light', style={'marginTop': '20px', 'textAlign': 'center', 'fontSize': '50px', 'color': 'white'}),
        html.P('Find your perfect movie match!', className='text-light', style={'textAlign': 'center', 'color': 'white'}),
        html.Div([
            dcc.Link(html.Button('User Recommendations', className='btn btn-primary btn-lg', style={'margin': '10px','backgroundColor': '#6eb92b','borderColor': '#6eb92b','boxShadow': '2px 2px 5px rgba(0, 0, 0, 0.2)','background-image': 'linear-gradient(to bottom right, #6eb92b, #5ca718)'} ), href='/users'),  # Apply button style
            dcc.Link(html.Button('Movie Recommendations', className='btn btn-secondary btn-lg',style={'color': '#333333','backgroundColor': '#f0f0f0'}), href='/items'),
        ], style={'textAlign': 'center', 'marginTop': '30px'}),
    ], style={'position': 'relative', 'zIndex': '2'}),
    html.Div(style={'position': 'fixed', 'width': '100%', 'height': '100%', 'backgroundImage': 'url("/assets/Picsart_24-05-22_18-13-21-630.jpg")', 'backgroundSize': 'cover', 'backgroundRepeat': 'no-repeat', 'backgroundPosition': 'center', 'top': '0', 'left': '0', 'zIndex': '1'}),
])


# Define the layout for the User page
user_page = html.Div([
    html.Div([
        html.A(html.I(className='fas fa-home', style={'fontSize': '36px', 'padding': '10px', 'color': '#6eb92b'}), href='/'),  # Home icon link with lime green color
        html.A(html.I(className='fas fa-film', style={'fontSize': '36px', 'padding': '10px', 'color': '#6eb92b'}), href='/items'),  # Film icon link with white color
    ], style={'position': 'absolute', 'top': '10px', 'left': '20px', 'zIndex': '2', 'display': 'flex', 'alignItems': 'center'}),
    html.Div([
        # html.H1([
        #     'Your Movie Journey: ',
        #     html.Span('Tailored Recommendations ', style={'color': '#6eb92b'}),
        #     ' Just for You'
        # ], className='text-light', style={'marginTop': '80px', 'textAlign': 'center', 'fontSize': '36px', 'color': 'white'}),
        html.H4('Personalized', style={'marginTop': '50px', 'marginBottom': '-20px', 'textAlign': 'center', 'color': '#6eb92b','fontSize': '30px'}),
        html.H1('Recommender', className='text-light', style={'marginTop': '0px', 'textAlign': 'center', 'color': 'white','fontSize': '50px'}),
        html.P(['Welcome to the ',
                html.Span('Limelight', style={'color': '#6eb92b'}),
                '! Your Personalized ',
                html.Span('Movie 🎥', style={'color': '#6eb92b'}),
                ' Picks Await! Select Your User ID and Set the Scene with Your Desired Recommendations.'
               ], className='text-light', style={'textAlign': 'center', 'color': 'grey', 'fontSize': '18px', 'margin': 'auto', 'maxWidth': '600px'}),
        html.Div([
        html.Label("Enter Your ID ", style={'color': 'white', 'marginTop': '20px', 'marginRight': '10px'}),
            dcc.Input(
                id='user-textbox',
                type='number',
                min=1,
                max=610,
                step=1,
                value=None,
                style={'width': '100px', 'backgroundColor': 'white', 'margin': 'auto', 'textAlign': 'left', 'borderRadius': '10px', 'padding-left': '10px'}
            ),
        ], style={'textAlign': 'center', 'marginTop': '20px'}),

        # Hidden div to store selected user ID
        html.Div(id='selected-user', style={'display': 'none'}),

        # Hidden div to store selected number of recommendations
        html.Div(id='selected-num', style={'display': 'none'}),

        # Previously watched movies with pagination
        html.Div(id='user-movies', style={'textAlign': 'center', 'marginTop': '20px'}),


        html.Div([
        html.Label("Select no. of recommendations", style={'color': 'white', 'marginTop': '20px', 'marginRight': '10px'}),
            dcc.Input(
                id='rec-dropdown',
                type='number',
                min=5,
                max=20,
                step=1,
                value=None,
                style={'width': '100px', 'backgroundColor': 'white', 'margin': 'auto', 'textAlign': 'left', 'borderRadius': '10px', 'padding-left': '10px'}
            ),
        ], style={'textAlign': 'center', 'marginTop': '20px'}),

        html.Div(id='rec-movies', style={'textAlign': 'center', 'marginTop': '20px'}),
        html.Div([
            html.I(className='fas fa-minus', id='rec-first-page', style={'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginRight': '5px', 'marginLeft': '5px', 'marginBottom': '15px', 'visibility': 'hidden'}),
            html.I(className='fas fa-minus', id='rec-second-page', style={'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginLeft': '5px', 'marginRight': '5px', 'marginBottom': '15px', 'visibility': 'hidden'}),
            html.I(className='fas fa-minus', id='rec-third-page', style={'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginLeft': '5px', 'marginRight': '5px', 'marginBottom': '15px', 'visibility': 'hidden'}),
            html.I(className='fas fa-minus', id='rec-fourth-page', style={'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginLeft': '5px', 'marginRight': '5px', 'marginBottom': '15px', 'visibility': 'hidden'}),
        ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center'})
    ], style={'position': 'relative', 'zIndex': '1'}),
    html.Div(style={'position': 'fixed', 'width': '100%', 'height': '100%', 'backgroundImage': 'url("/assets/Picsart_24-05-22_18-13-21-630.jpg")', 'backgroundSize': 'cover', 'backgroundRepeat': 'no-repeat', 'backgroundPosition': 'center', 'top': '0', 'left': '0', 'zIndex': '0'})
])



# Define the layout for the Item (Movie) page
item_page = html.Div([
    html.Div([
        html.A(html.I(className='fas fa-home', style={'fontSize': '36px', 'padding': '10px', 'color': '#6eb92b'}), href='/'),
        html.A(html.I(className='fas fa-user', style={'fontSize': '36px', 'padding': '10px', 'color': '#6eb92b'}), href='/users'),
    ], style={'position': 'absolute', 'top': '10px', 'left': '20px', 'zIndex': '2', 'display': 'flex', 'alignItems': 'center'}),
    html.Div([
        html.H4('Movie', style={'marginTop': '50px', 'marginBottom': '-20px', 'textAlign': 'center', 'color': '#6eb92b','fontSize': '30px'}),
        html.H1('Recommender', className='text-light', style={'marginTop': '0px', 'textAlign': 'center', 'color': 'white','fontSize': '50px'}),
       html.Div([
    html.Span("Lights, camera, ", style={'color': 'white', 'fontSize': '18px'}),
    html.Span("action!🎬", style={'color': '#6eb92b', 'fontSize': '18px'}),
    html.Span(" Choose your movie, then dial in your recommendations: by rating or genre. Let's roll!", style={'color': 'white', 'fontSize': '18px'}),
], style={'textAlign': 'center', 'marginBottom': '30px', 'maxWidth': '600px', 'margin': 'auto'}),

        dcc.Dropdown(
            id='movie-dropdown',
            options=[{'label': title, 'value': title} for title in movies_genres_df_new['title']],
            value=None,
            placeholder="Select a movie...",
            style={'width': '300px', 'backgroundColor': 'white', 'margin': 'auto', 'textAlign': 'left', 'borderRadius': '10px', 'marginTop': '20px'}
        ),
        html.Div(id="selected-movie"),
        html.Div(id='movie-info-output', style={'color': 'white'}),
        html.Div([
            html.Label("Recommendations Based On", style={'color': 'white', 'marginRight': '10px'}),
            dcc.RadioItems(
                id='recommendation-type',
                options=[
                    {'label': ' Genre', 'value': 'genre'},
                    {'label': ' Rating', 'value': 'rating'}
                ],
                value=None,
                labelStyle={'display': 'inline-block', 'margin': '10px', 'color': 'white'}
            ),
        ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center'}),
        html.Label("Number of Recommendations", style={'color': 'white', 'marginTop': '20px', 'marginRight': '10px'}),
        dcc.Input(
            id='num-recommendations',
            type='number',
            min=5,
            max=20,
            step=1,
            value=5,
            style={'width': '50px', 'backgroundColor': 'white', 'margin': 'auto', 'textAlign': 'left', 'borderRadius': '10px', 'padding-left': '10px'}
        ),
        html.Div([
            html.Ul(
                id='similar-movies-list',
                style={'list-style-type': 'none', 'display': 'flex', 'overflowX': 'auto', 'padding': '0', 'margin': '0', 'color': 'white', 'width': 'fit-content', 'position': 'relative', 'left': '-10px'}
            ),
        ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'marginTop': '20px'}),
        html.Div([
            html.I(className='fas fa-minus', id='first-page', style={'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginRight': '5px', 'marginLeft': '5px', 'marginBottom': '15px', 'visibility': 'hidden'}),
            html.I(className='fas fa-minus', id='second-page', style={'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginLeft': '5px', 'marginRight': '5px', 'marginBottom': '15px', 'visibility': 'hidden'}),
            html.I(className='fas fa-minus', id='third-page', style={'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginLeft': '5px', 'marginRight': '5px', 'marginBottom': '15px', 'visibility': 'hidden'}),
            html.I(className='fas fa-minus', id='fourth-page', style={'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginLeft': '5px', 'marginRight': '5px', 'marginBottom': '15px', 'visibility': 'hidden'}),
        ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center'})
    ], style={'position': 'relative', 'zIndex': '1', 'textAlign': 'center'}),
    html.Div(style={'position': 'fixed', 'width': '100%', 'height': '100%', 'backgroundImage': 'url("./assets/Picsart_24-05-22_18-13-21-630.jpg")', 'backgroundSize': 'cover', 'backgroundRepeat': 'no-repeat', 'backgroundPosition': 'center', 'top': '0', 'left': '0', 'zIndex': '0'}),
])


# Define the main layout with dcc.Location to track the URL
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    html.Div(id='page-content')
])

# Callback to update the page content based on the URL
@callback(Output('page-content', 'children'), [Input('url', 'pathname')])
def display_page(pathname):
    if pathname == '/users':
        return user_page
    elif pathname == '/items':
        return item_page
    else:
        return main_page

# Callback to store the selected user value in a hidden div
@app.callback(Output('selected-user', 'children'), [Input('user-textbox', 'value')])
def store_selected_user(value):
    return value

@app.callback(Output('selected-num', 'children'), [Input('rec-dropdown', 'value')])
def store_selected_rec_num(rec):
    return rec

# Function to fetch movie details
def fetch_movie_details(imdb_id, api_key):
    url = f'http://www.omdbapi.com/?i={imdb_id}&apikey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        movie_data = response.json()
        return {
            'title': movie_data.get('Title'),
            'release_year': movie_data.get('Year'),
            'duration': movie_data.get('Runtime'),  # Fetch duration of the movie
            'image': movie_data.get('Poster')
        }
    else:
        return None


# Callback to update user movies based on selected user ID
@app.callback(Output('user-movies', 'children'), [Input('user-textbox', 'value')])
def display_user_movies(user_id):
    if not user_id:
        return html.Div()  # Return an empty div if no user is selected

    ratings_df = pd.read_csv('ratings.csv')
    links_df = pd.read_csv('/content/Links_processed.csv')
    links_df = links_df.drop(columns=['tmdbId'])
    user_ratings = ratings_df[ratings_df['userId'] == user_id]['movieId']
    filtered_links = links_df[links_df['movieId'].isin(user_ratings)]
    selected_columns = filtered_links[['movieId', 'imdbId']]

    movie_details_list = []
    for imdb_id in selected_columns['imdbId']:
        details = fetch_movie_details(imdb_id, api_key)
        if details:
            movie_details_list.append(details)

    # Generate movie details layout
    movie_details_layout = html.Div([
    html.H2("Previously Watched", style={'color': 'white', 'textAlign': 'center', 'fontSize': '24px', 'marginBottom': '20px'}),
    html.Div([
        html.Div([
            html.Div([
                html.Img(src=movie['image'], style={'width': '200px', 'height': '300px'}),
                html.P(movie['title'], style={'color': 'white', 'width': '200px', 'textAlign': 'center', 'whiteSpace': 'normal', 'overflowWrap': 'break-word'}),
                html.P(f"{movie['release_year']}", style={'color': 'white', 'textAlign': 'center'}),
                # html.P(f"Duration: {movie['duration']}", style={'color': 'white', 'textAlign': 'center'})
            ], style={'margin': '10px', 'maxWidth': '200px', 'textAlign': 'center'})
            for movie in movie_details_list
        ], style={'display': 'flex', 'overflowX': 'auto', 'whiteSpace': 'nowrap', 'overflowY': 'hidden','color': 'red'})
    ], style={'overflowX': 'hidden', 'color': 'red'})  # Adding color style for overflow

], style={'marginTop': '20px'})

    return movie_details_layout




@app.callback(
    [Output('rec-movies', 'children'),
     Output('rec-first-page', 'style'),
     Output('rec-second-page', 'style'),
     Output('rec-third-page', 'style'),
     Output('rec-fourth-page', 'style')],
    [Input('user-textbox', 'value'),
     Input('rec-dropdown', 'value'),
     Input('rec-first-page', 'n_clicks'),
     Input('rec-second-page', 'n_clicks'),
     Input('rec-third-page', 'n_clicks'),
     Input('rec-fourth-page', 'n_clicks')],
    [State('rec-movies', 'children')]
)
def display_rec(user_id, n_recommendations, first_clicks, second_clicks, third_clicks, fourth_clicks, current_similar_movies):
    if user_id is None or n_recommendations is None:
        return html.Div(), {'visibility': 'hidden'}, {'visibility': 'hidden'}, {'visibility': 'hidden'}, {'visibility': 'hidden'}

    links_df = pd.read_csv('/content/Links_processed.csv')
    links_df = links_df.drop(columns=['tmdbId'])

    # Convert n_recommendations to integer
    n_recommendations = int(n_recommendations)

    recommendations = model.recommend_user(user=user_id, n_rec=n_recommendations)
    recommended_movie_ids = recommendations[user_id].tolist()
    recommended_links = links[links['movieId'].isin(recommended_movie_ids)]
    recommended_links['movieId'] = pd.Categorical(recommended_links['movieId'], categories=recommended_movie_ids, ordered=True)
    recommended_links = recommended_links.sort_values('movieId')
    recommended_columns = recommended_links[['movieId', 'imdbId']]
    recommended_columns = recommended_columns.reset_index(drop=True)

    recommended_movie_details_list = []
    for _, row in recommended_columns.iterrows():
        imdb_id = row['imdbId']
        details = fetch_movie_details(imdb_id, api_key)
        if details:
            recommended_movie_details_list.append(details)


    movies_per_page = 5
    total_pages = (len(recommended_movie_details_list) + movies_per_page - 1) // movies_per_page

    # Initialize current_page
    current_page = 0

    # Update current_page based on page clicks
    ctx = dash.callback_context
    if ctx.triggered:
        trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if trigger_id == 'rec-first-page' and first_clicks:
            current_page = 0
        elif trigger_id == 'rec-second-page' and second_clicks:
            current_page = 1
        elif trigger_id == 'rec-third-page' and third_clicks:
            current_page = 2
        elif trigger_id == 'rec-fourth-page' and fourth_clicks:
            current_page = 3

    # Calculate start and end indices of the current page
    start_idx = current_page * movies_per_page
    end_idx = min(start_idx + movies_per_page, len(recommended_movie_details_list))

    # Define styles
    first_page_style = {'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginRight': '5px', 'marginLeft': '5px', 'marginBottom': '15px'}
    second_page_style = {'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginRight': '5px', 'marginLeft': '5px', 'marginBottom': '15px'}
    third_page_style = {'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginRight': '5px', 'marginLeft': '5px', 'marginBottom': '15px'}
    fourth_page_style = {'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginRight': '5px', 'marginLeft': '5px', 'marginBottom': '15px'}

    # Determine which circles to show and update colors based on the current page
    if total_pages == 1:
        first_page_style['visibility'] = 'visible'
        first_page_style['color'] = '#6BBB25'
        second_page_style['visibility'] = 'hidden'
        third_page_style['visibility'] = 'hidden'
        fourth_page_style['visibility'] = 'hidden'
        first_page_style['marginLeft'] = '80px'

    elif total_pages == 2:
        first_page_style['visibility'] = 'visible'
        second_page_style['visibility'] = 'visible'
        third_page_style['visibility'] = 'hidden'
        fourth_page_style['visibility'] = 'hidden'
        if current_page == 0:
            first_page_style['color'] = '#6BBB25'
        elif current_page == 1:
            second_page_style['color'] = '#6BBB25'
        first_page_style['marginLeft'] = '60px'
        second_page_style['marginLeft'] = '5px'

    elif total_pages == 3:
        first_page_style['visibility'] = 'visible'
        second_page_style['visibility'] = 'visible'
        third_page_style['visibility'] = 'visible'
        fourth_page_style['visibility'] = 'hidden'
        if current_page == 0:
            first_page_style['color'] = '#6BBB25'
        elif current_page == 1:
            second_page_style['color'] = '#6BBB25'
        elif current_page == 2:
            third_page_style['color'] = '#6BBB25'
        first_page_style['marginLeft'] = '40px'
        second_page_style['marginLeft'] = '5px'
        third_page_style['marginLeft'] = '5px'

    elif total_pages >= 4:
        first_page_style['visibility'] = 'visible'
        second_page_style['visibility'] = 'visible'
        third_page_style['visibility'] = 'visible'
        fourth_page_style['visibility'] = 'visible'
        if current_page == 0:
            first_page_style['color'] = '#6BBB25'
        elif current_page == 1:
            second_page_style['color'] = '#6BBB25'
        elif current_page == 2:
            third_page_style['color'] = '#6BBB25'
        elif current_page == 3:
            fourth_page_style['color'] = '#6BBB25'

    # Create a list of recommended movies with <li> elements
    recommended_movies_list = html.Ul(
        [
            html.Li(
                [
                    html.Div(
                        [
                            html.Img(src=movie['image'], style={'width': '200px', 'height': '300px'}),
                            html.P(movie['title'], style={'color': 'white', 'width': '200px', 'textAlign': 'center', 'whiteSpace': 'normal', 'overflowWrap': 'break-word'}),
                            html.P(f"{movie['release_year']}", style={'color': 'white', 'textAlign': 'center'}),
                        ], style={'margin': '10px', 'maxWidth': '200px', 'textAlign': 'center'}
                    )
                ]
            ) for movie in recommended_movie_details_list[start_idx:end_idx]
        ],
        style={'list-style-type': 'none', 'display': 'flex', 'flexWrap': 'wrap', 'padding': '0', 'margin': '0', 'justifyContent': 'center'}
    )

    return recommended_movies_list, first_page_style, second_page_style, third_page_style, fourth_page_style



@app.callback(
    Output('movie-info-output', 'children'),
    [Input('movie-dropdown', 'value')]
)
def update_movie_info(selected_title):
    if selected_title:
        movie_id = movie_ratings.loc[movie_ratings['title'] == selected_title, 'movieId'].values[0]
        average_rating = movie_ratings.loc[movie_ratings['movieId'] == movie_id, 'rating'].mean()
        genres = movie_ratings.loc[movie_ratings['title'] == selected_title, 'genres'].values[0]
        genres_list = eval(genres)
        genres_string = ', '.join(genres_list)
        year = movie_ratings.loc[movie_ratings['title'] == selected_title, 'year'].values[0]

        imdb_id = links.loc[links['movieId'] == movie_id, 'imdbId'].values[0]
        poster_url = fetch_movie_poster(imdb_id)

        # Calculate star ratings
        full_stars = int(average_rating)
        half_star = 1 if (average_rating - full_stars) > 0.1 else 0
        empty_stars = 5 - full_stars - half_star

        # Create star elements
        stars = []
        for _ in range(full_stars):
            stars.append(html.I(className='fas fa-star', style={'color': 'gold', 'marginRight': '5px'}))
        if half_star:
            stars.append(html.I(className='fas fa-star-half-alt', style={'color': 'gold', 'marginRight': '5px'}))
        for _ in range(empty_stars):
            stars.append(html.I(className='far fa-star', style={'color': 'gold', 'marginRight': '5px'}))

        movie_info = [
            html.Img(src=poster_url, style={'width': '200px', 'height': '300px', 'margin-top': '20px', 'margin-bottom': '20px'}),
            html.P(f"{year}", style={'color': 'white'}),
            html.P(f"Rating: {average_rating:.2f}", style={'color': 'white'}),
            html.Div(stars, style={'display': 'flex', 'justifyContent': 'center', 'marginBottom': '20px'}),
            html.P(f"{genres_string}", style={'color': 'white'})
        ]
        return html.Div(movie_info)
    else:
        return html.Div("No movie selected", style={'color': 'white'})


@app.callback(
    Output('similar-movies-list', 'children'),
    Output('first-page', 'style'),
    Output('second-page', 'style'),
    Output('third-page', 'style'),
    Output('fourth-page', 'style'),
    [Input('movie-dropdown', 'value'),
     Input('recommendation-type', 'value'),
     Input('num-recommendations', 'value'),
     Input('first-page', 'n_clicks'),
     Input('second-page', 'n_clicks'),
     Input('third-page', 'n_clicks'),
     Input('fourth-page', 'n_clicks')],
    [State('similar-movies-list', 'children')]
)
def update_similar_movies(movie_title, recommendation_type, num_recommendations, first_clicks, second_clicks, third_clicks, fourth_clicks, current_similar_movies):
    if not movie_title:
        return [], {'visibility': 'hidden'}, {'visibility': 'hidden'}, {'visibility': 'hidden'}, {'visibility': 'hidden'}

    similar_movies = []
    if recommendation_type == 'genre':
        similar_movies = top_similar_movies_based_on_genres(movies_genres_df_new, movie_title, num_recommendations)
    elif recommendation_type == 'rating':
        similar_movies = top_similar_movies_based_on_rating(movie_ratings_df, movie_title, num_recommendations)

    similar_movies_with_posters = []
    for movie_title in similar_movies:
        movie_id = movie_ratings.loc[movie_ratings['title'] == movie_title, 'movieId'].values[0]
        imdb_id = links.loc[links['movieId'] == movie_id, 'imdbId'].values[0]
        poster_url = fetch_movie_poster(imdb_id)
        movie_to_be_shown = fetch_movie_title(imdb_id)
        similar_movies_with_posters.append(html.Div([
            html.Img(src=poster_url, style={'width': '200px', 'height': '300px'}),
            html.P(movie_to_be_shown, style={'color': 'white', 'width': '200px'})
        ], style={'display': 'inline-block', 'marginRight':'20px'}))

    movies_per_page = 5
    total_pages = (len(similar_movies_with_posters) + movies_per_page - 1) // movies_per_page

    # Initialize current_page
    current_page = 0

    # Update current_page based on page clicks
    ctx = dash.callback_context
    if ctx.triggered:
        trigger_id = ctx.triggered[0]['prop_id'].split('.')[0]
        if trigger_id == 'first-page' and first_clicks:
            current_page = 0
        elif trigger_id == 'second-page' and second_clicks:
            current_page = 1
        elif trigger_id == 'third-page' and third_clicks:
            current_page = 2
        elif trigger_id == 'fourth-page' and fourth_clicks:
            current_page = 3

    # Calculate start and end indices of the current page
    start_idx = current_page * movies_per_page
    end_idx = min(start_idx + movies_per_page, len(similar_movies_with_posters))

    # Define styles
    first_page_style = {'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginRight': '5px', 'marginLeft': '5px', 'marginBottom': '15px'}
    second_page_style = {'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginRight': '5px', 'marginLeft': '5px', 'marginBottom': '15px'}
    third_page_style = {'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginRight': '5px', 'marginLeft': '5px', 'marginBottom': '15px'}
    fourth_page_style = {'fontSize': '20px', 'color': '#555', 'cursor': 'pointer', 'marginRight': '5px', 'marginLeft': '5px', 'marginBottom': '15px'}

    # Determine which circles to show and update colors based on the current page
    if total_pages == 1:
        first_page_style['visibility'] = 'visible'
        first_page_style['color'] = '#6BBB25'
        second_page_style['visibility'] = 'hidden'
        third_page_style['visibility'] = 'hidden'
        fourth_page_style['visibility'] = 'hidden'
        first_page_style['marginLeft'] = '60px'

    elif total_pages == 2:
        first_page_style['visibility'] = 'visible'
        second_page_style['visibility'] = 'visible'
        third_page_style['visibility'] = 'hidden'
        fourth_page_style['visibility'] = 'hidden'
        if current_page == 0:
            first_page_style['color'] = '#6BBB25'
        elif current_page == 1:
            second_page_style['color'] = '#6BBB25'
        first_page_style['marginLeft'] = '40px'
        second_page_style['marginLeft'] = '5px'

    elif total_pages == 3:
        first_page_style['visibility'] = 'visible'
        second_page_style['visibility'] = 'visible'
        third_page_style['visibility'] = 'visible'
        fourth_page_style['visibility'] = 'hidden'
        if current_page == 0:
            first_page_style['color'] = '#6BBB25'
        elif current_page == 1:
            second_page_style['color'] = '#6BBB25'
        elif current_page == 2:
            third_page_style['color'] = '#6BBB25'
        first_page_style['marginLeft'] = '20px'
        second_page_style['marginLeft'] = '5px'
        third_page_style['marginLeft'] = '5px'

    elif total_pages >= 4:
        first_page_style['visibility'] = 'visible'
        second_page_style['visibility'] = 'visible'
        third_page_style['visibility'] = 'visible'
        fourth_page_style['visibility'] = 'visible'
        if current_page == 0:
            first_page_style['color'] = '#6BBB25'
        elif current_page == 1:
            second_page_style['color'] = '#6BBB25'
        elif current_page == 2:
            third_page_style['color'] = '#6BBB25'
        elif current_page == 3:
            fourth_page_style['color'] = '#6BBB25'

    return similar_movies_with_posters[start_idx:end_idx], first_page_style, second_page_style, third_page_style, fourth_page_style



if __name__ == '__main__':
    app.run_server(port=8051, jupyter_mode='external')


Dash app running on:


<IPython.core.display.Javascript object>